@author: J. D. Camacho

# Notebook to search the best sub-model for a given dataset

This notebook will compute the best architecture and hyper-parameters for a model using the hyperband method

In [1]:
# Import basic modules
import os
os.environ['PATH'] += os.environ['CUDA_PATH_V11_0']

import tensorflow as tf
import keras_tuner as kt

from keras_autocompressor.hypermodels import HyperAutoCompressMobileNetV2

In [2]:
# Load DS
ds_path = os.environ['DS_PATH']
ds_folder = os.path.join(ds_path, 'flowers')

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=ds_folder,
    label_mode='categorical',
    seed=1234, 
    image_size=(224,224),
    batch_size=32,
    validation_split=0.3,
    subset='training',
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=ds_folder,
    label_mode='categorical',
    seed=1234, 
    image_size=(224,224),
    batch_size=32,
    validation_split=0.3,
    subset='validation',
)

lambda_func = lambda x, y: (
    tf.keras.applications.mobilenet_v2.preprocess_input(x), y
)

train_ds = train_ds.map(lambda_func)
val_ds = val_ds.map(lambda_func)

Found 4317 files belonging to 5 classes.
Using 3022 files for training.
Found 4317 files belonging to 5 classes.
Using 1295 files for validation.


In [3]:
# Hyper model
HyperModel = HyperAutoCompressMobileNetV2()

# Create hyperparameters object and to register in advance building data
hp = kt.HyperParameters()
hp.Fixed('n_classes', 5)

mytuner = kt.Hyperband(HyperModel,
                       max_epochs=100,
                       objective=kt.Objective("val_acc_comp", direction="max"),
                       directory='skipp_tuner_v4',
                       project_name='mobilenetv2_flowers',
                       hyperparameters=hp,)

INFO:tensorflow:Reloading Oracle from existing project skipp_tuner_v4\mobilenetv2_flowers\oracle.json
INFO:tensorflow:Reloading Tuner from skipp_tuner_v4\mobilenetv2_flowers\tuner0.json


In [4]:
mytuner.search(train_ds, validation_data=val_ds, epochs=10)

Trial 54 Complete [00h 00m 52s]
val_acc_comp: 0.7323843240737915

Best val_acc_comp So Far: 0.7823328375816345
Total elapsed time: 00h 46m 47s

Search: Running Trial #55

Hyperparameter    |Value             |Best Value So Far 
n_classes         |5                 |5                 
base_model        |44,143            |89,151            
dropout_1         |0.3               |0.2               
fc_units          |12                |28                
dropout_2         |0.1               |0.2               
lr                |0.00070318        |0.0062102         
tuner/epochs      |2                 |2                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |4                 |4                 
tuner/round       |0                 |0                 

Epoch 1/2
19/95 [=====>........................] - ETA: 13s - loss: 1.5691 - accuracy: 0.2753 - acc_comp: 0.3357

KeyboardInterrupt: 

In [5]:
best_hp = mytuner.get_best_hyperparameters()[0]

In [6]:
best_hp.values

{'n_classes': 5,
 'base_model': '89,151',
 'dropout_1': 0.2,
 'fc_units': 28,
 'dropout_2': 0.2,
 'lr': 0.006210193779256237,
 'tuner/epochs': 2,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 4,
 'tuner/round': 0}

In [7]:
best_model = mytuner.get_best_models()[0]

In [4]:
for sub_model in mytuner.get_best_models(15):
    metrics = sub_model.evaluate(val_ds, verbose=0)
    print('val_accuracy: {}  -- params: {}'.format(metrics[1], sub_model.count_params()))

val_accuracy: 0.8077220320701599  -- params: 733101
val_accuracy: 0.7976834177970886  -- params: 656589
val_accuracy: 0.7907335758209229  -- params: 600653
val_accuracy: 0.8393822312355042  -- params: 1053229
val_accuracy: 0.785328209400177  -- params: 590365
val_accuracy: 0.785328209400177  -- params: 600653
val_accuracy: 0.816216230392456  -- params: 1042941
val_accuracy: 0.8270270228385925  -- params: 1200733
val_accuracy: 0.7436293363571167  -- params: 501741
val_accuracy: 0.8100386261940002  -- params: 1231597
val_accuracy: 0.7961390018463135  -- params: 1124589
val_accuracy: 0.7868725657463074  -- params: 1042941
val_accuracy: 0.7806949615478516  -- params: 1058365
val_accuracy: 0.7760617733001709  -- params: 1023005
val_accuracy: 0.7359073162078857  -- params: 733101


In [8]:
best_model.summary()

Model: "autosearch"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
_________________________________________________________________________________________

In [9]:
best_model.evaluate(val_ds)

41/41 [==============================] - 8s 153ms/step - loss: 0.5541 - accuracy: 0.7987 - acc_comp: 0.7751


[0.5439008474349976, 0.8077220320701599, 0.7823328375816345]

In [10]:
mytuner.results_summary()

Results summary
Results in skipp_tuner_v4\mobilenetv2_flowers
Showing 10 best trials
Objective(name='val_acc_comp', direction='max')
Trial summary
Hyperparameters:
n_classes: 5
base_model: 89,151
dropout_1: 0.2
fc_units: 28
dropout_2: 0.2
lr: 0.006210193779256237
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 4
tuner/round: 0
Score: 0.7823328375816345
Trial summary
Hyperparameters:
n_classes: 5
base_model: 80,151
dropout_1: 0.2
fc_units: 12
dropout_2: 0.1
lr: 0.001442914665091062
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 4
tuner/round: 0
Score: 0.780965268611908
Trial summary
Hyperparameters:
n_classes: 5
base_model: 71,151
dropout_1: 0.0
fc_units: 12
dropout_2: 0.4
lr: 0.0029846580829526737
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 4
tuner/round: 0
Score: 0.7802767753601074
Trial summary
Hyperparameters:
n_classes: 5
base_model: 115,151
dropout_1: 0.1
fc_units: 28
dropout_2: 0.1
lr: 0.001787094577786772
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracke